In [1]:
from langchain_community.retrievers import BM25Retriever
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.retrievers import EnsembleRetriever
import transformers
import torch

c:\Users\woojin\anaconda3\envs\assistant\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

data = [
    {
        "기업명": "삼성전자",
        "날짜": "2024-03-02",
        "문서 카테고리": "인수합병",
        "요약": "삼성전자가 HVAC(냉난방공조) 사업 인수를 타진 중이며, 이는 기존 가전 사업의 약점 보완을 목적으로 한다.",
        "주요 이벤트": ["기업 인수합병"]
    },
    {
        "기업명": "삼성전자",
        "날짜": "2024-03-24",
        "문서 카테고리": "인수합병",
        "요약": "테스트 하나 둘 셋",
        "주요 이벤트": ["신제품 출시"]
    },
    {
        "기업명": "현대차",
        "날짜": "2024-04-02",
        "문서 카테고리": "인수합병",
        "요약": "삼성전자가 HVAC(냉난방공조) 사업 인수를 타진 중이며, 이는 기존 가전 사업의 약점 보완을 목적으로 한다.",
        "주요 이벤트": ["기업 인수합병", "신제품 출시"]
    },        
]

doc_list = [item['요약'] for item in data]

bm25_retriever = BM25Retriever.from_texts(
    doc_list, metadatas=[{'source': i} for i in range(len(data))]
)
bm25_retriever.k = 1

# OpenAI embedding -> sentence-transformers embedding
embedding = SentenceTransformerEmbeddings(model_name="distiluse-base-multilingual-cased-v1")
faiss_vectorstore = FAISS.from_texts(
    doc_list, embedding, metadatas=[{'source': i} for i in data]
)
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={'k':1})

ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever],
    weights=[0.5, 0.5]  # openai embedding 보다 성능이 낮아지므로 bm25 가중치 높임
)

# OpenAI LLM model -> Huggingface 42dot sLLM model
model_name = "42dot/42dot_LLM-SFT-1.3B"

pipleline = transformers.pipeline(
    'text-generation',
    device=device,
    model=model_name,
    model_kwargs={"torch_dtype": torch.float16}
)

pipleline.model.eval()

def retrieve(query):
    ensemble_docs = ensemble_retriever.invoke(query)
    return ensemble_docs

def sllm_generate(query):
    
    answer = pipleline(
        query,
        max_new_tokens=100,
        do_sample=True,
        temperature=0.5,
        top_p=0.9
    )
    return answer[0]['generated_text'][len(query):]

def prompt_and_generate(query, docs):
    
    prompt = f"""아래 질문을 기반으로 검색된 뉴스를 참고하여 질문에 대한 답변을 생성하시오.
    질문: {query}
    """
    
    for i in range(len(docs)):
        prompt += f"뉴스{i+1}\n"
        prompt += f"요약: {docs[i].page_content}\n"
        prompt += "\n"
    prompt += "답변:"
    
    answer = sllm_generate(prompt)
    return answer

C:\Users\woojin\AppData\Local\Temp\ipykernel_10356\2134716307.py:35: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embedding = SentenceTransformerEmbeddings(model_name="distiluse-base-multilingual-cased-v1")
c:\Users\woojin\anaconda3\envs\assistant\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
c:\Users\woojin\anaconda3\envs\assistant\Lib\site-packages\huggingface_h

In [3]:
query = "삼성전자가 인수하는 기업은?"
retrieved = [doc for doc in retrieve(query)]
answer = prompt_and_generate(query, retrieved)
print(answer)

c:\Users\woojin\anaconda3\envs\assistant\Lib\site-packages\transformers\models\llama\modeling_llama.py:660: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


 삼성전자가 인수하는 기업은 테스트 하나 둘 셋입니다.
